In [11]:
import numpy as np
from scipy.special import expit

def normalize_data(tx):
    # normalizing data by features and add bias
    input_data = np.zeros_like(tx, dtype = np.float)
    for i in range(input_data.shape[1]):
      input_data[:,i] = tx[:,i] -  np.mean(tx[:,i])
      input_data[:,i] = tx[:,i]/np.std(tx[:,i])
    return  np.concatenate((np.ones((tx.shape[0],1)), input_data), axis = 1)
def check_linear_dependenece(tx):
    #check if there are lineary dependent cols
    print("Feature matrix rank is ", np.linalg.matrix_rank(matrix, tol = 1e-5))
    for i in range(matrix.shape[1]):
      for j in range(matrix.shape[1]):
        if i != j:
            inner_product = np.inner(
                matrix[:,i],
                matrix[:,j]
            )
            norm_i = np.linalg.norm(matrix[:,i])
            norm_j = np.linalg.norm(matrix[:,j])

            if np.abs(inner_product - norm_j * norm_i) < 1E-5:
                print( 'Dependent rows #{} and #{}'.format(i, j) )
                print( 'I: ', matrix[:,i])
                print('J: ', matrix[:,j])
                print( 'Prod: ', inner_product)
                print('Norm i: ', norm_i)
                print('Norm j: ', norm_j)
def remove_outliers(X, y, threshold=5.0):
    idx = (np.abs(X) < threshold).all(axis=1)
    return X[idx], y[idx], idx
def replace_missing_values(x):
  nr = x.shape[0]
  mean = [np.mean(x[:,i]) for i in range(x.shape[1])]
  for i in range(nr):
    for ind in np.where(x[i] == -999)[0]:
        x[i,ind] = mean[ind]
  return x
def delete_missing_values(x, y):
  row_nmb = []
  nr = x.shape[0]
  for i in range(nr):
    if np.sum(x[i] == -999.) > 0:
        row_nmb.append(i)
  print("% of axis to be deleted is ", len(row_nmb)/nr)
  return np.delete(x, row_nmb, axis = 0), np.delete(y, row_nmb, axis = 0)
  """
  dist = compute_pairwise_dist_of_rows(x)
  print("Searching for missing values")
  nr, nc = x.shape
  for i in range(nr):
    if (x[i] == -999).shape[0] > 0:
      where = np.where(i == -999)
      for j in where:
        x[i,j] = x[np.where(dist == np.amin(dist[i]))[0], k ] + np.random.randn()*np.std(x[:,k])/2.
  return normalize_data(x)
    """
def MSE(y, tx, w):
    return np.average((y - np.dot(tx,w)) ** 2)
def MSE(y, tx, w):
    return np.average((y - np.dot(tx,w)) ** 2)
def MSE_long(y, tx, w): #is made because usual computation causes RAM overflow
    mse = 0.
    len_y = len(y)
    for i in range(len_y):
      mse += (y[i] - np.dot(tx[i], w)) **2
    return mse/np.float(len_y)
def MAE(y,tx, w):
    return np.average(np.abs(y - np.dot(tx,w)))
def RMSE(y,tx,w):
    return np.sqrt(2.*MSE_long(y,tx,w))
def loss_function(y,tx,w):
    return MSE_long(y,tx,w)

def compute_gradient(y, tx, w):
    """Compute the gradient."""
    tmp = np.dot(tx,w)
    tmpp = y + (tmp<0).astype(np.float) - (tmp>=0).astype(np.float)
    return -np.dot(tx.T, tmpp)/float(y.shape[0])
def compute_stoch_gradient(y, tx, w):
    """Compute a stochastic gradient from just few examples n and their corresponding y_n labels."""
    tmp = np.dot(tx,w)
    tmpp = y + (tmp<0).astype(np.float) - (tmp>=0).astype(np.float)
    return np.expand_dims(np.average(-np.dot(tx.T, tmpp), axis = 1), axis =1)
def batch_iter(y, tx, batch_size = 10, shuffle=True):
    data_size = len(y)
    if shuffle:
        shuffle_indices = np.random.permutation(np.arange(data_size))
        shuffled_y = y[shuffle_indices]
        shuffled_tx = tx[shuffle_indices]
    else:
        shuffled_y = y
        shuffled_tx = tx
    num_batches = int(data_size/batch_size)
    for batch_num in range(num_batches):
        start_index = batch_num * batch_size
        end_index = min((batch_num + 1) * batch_size, data_size)
        if start_index != end_index:
            yield shuffled_y[start_index:end_index], shuffled_tx[start_index:end_index]
          

def least_squares_GD(y, tx, initial_w, max_iters, gamma):
    """Gradient descent algorithm."""
    w = initial_w
    r = 0.75
    for n_iter in range(max_iters):
        gamma = 1./pow(np.float(1+n_iter), r)
        w = w - gamma*compute_gradient(y, tx, w)
        if gamma <= 1e-5:
          break
        if n_iter%100 == 0:
          print("Making {} iteration, {} iterations remain".format(n_iter, max_iters - n_iter))
    print("DG returnind results, gamma = {}".format(gamma))
    return w,loss_function(y,tx,w)  
    
def least_squares_SGD(y, tx, initial_w, max_iters, gamma): #using batch
    """Stochastic gradient descent algorithm."""
    
    w = initial_w
    n_iter = 0
    r = 0.75
    while(n_iter < max_iters):
      for b_y,b_tx in batch_iter(y, tx, batch_size = 1, shuffle=True):
          gamma = 1./pow(np.float(1+n_iter), r)
          w = w - gamma*compute_stoch_gradient(b_y, b_tx, w)
          n_iter+=1
          if n_iter%10000 == 0:
            print( "Iteration #{}, gamma = {}".format(n_iter, gamma))
          if n_iter >= max_iters or gamma <= 1e-5:
              break
    
    print("SDG returnind results, gamma = {}".format(gamma))
    return w,loss_function(y,tx,w)

def least_squares(y, tx):
    w0 = np.dot(np.linalg.inv(np.dot(tx.T, tx)), np.dot(tx.T, y))
    return w0, loss_function(y,tx,w0)

def ridge_regression(y, tx, lambda_):
    N, d = tx.shape
    w0 = np.dot(np.linalg.inv(np.dot(tx.T, tx) + np.float(2*N*lambda_)*
                              np.ones((d,d), dtype = np.float)), np.dot(tx.T, y))
    return w0, loss_function(y,tx,w0)

def logistic_regression(y, tx, initial_w, max_iters, gamma, hessian = True):
    def sigmoid(x):
        return 1./(1.+ np.exp(-x))
    def log_loss(y, tx, w):
        """compute the cost by negative log likelihood."""
        tmp = 0.
        print("Calculating log loss")
        for i in range(len(y)):
            tmp += (np.log(1+ np.exp(tx[i] @ w)) - y[i]*(tx[i] @ w))
        return tmp
    def calculate_stoch_gradient(y_i, tx_i, w):
        return tx_i.T * (sigmoid(np.dot(tx_i, w)) - y_i)
    r = 0.75
    n_iter = 0
    w = initial_w
    while(n_iter < max_iters):
        for b_y,b_tx in batch_iter(y, tx, batch_size = 1, shuffle=True):
            gamma = 1./pow(np.float(1+n_iter), r)
            if n_iter%10000 == 0:
                print( "Iteration #{}, gamma = {}".format(n_iter, gamma))
            w -= gamma*calculate_stoch_gradient(b_y, b_tx, w)
            n_iter +=1
            if n_iter >= max_iters or gamma <= 1e-5:
                break
        if n_iter >= max_iters or gamma <= 1e-5:
            break
    return w, log_loss(y,tx,w)
def reg_logistic_regression(y, tx, lambda_ , initial_w, max_iters, gamma):
    def sigmoid(x):
        return 1./(1.+ np.exp(-x))
    def log_loss(y, tx, w):
        """compute the cost by negative log likelihood."""
        tmp = 0.
        print("Calculating log loss")
        for i in range(len(y)):
            tmp += (np.log(1+ np.exp(tx[i] @ w)) - y[i]*(tx[i] @ w))
        return tmp
    def calculate_stoch_gradient(y_i, tx_i, w):
        return tx_i.T * (sigmoid(np.dot(tx_i, w)) - y_i)
    r = 0.75
    n_iter = 0
    w = initial_w
    while(n_iter < max_iters):
        for b_y,b_tx in batch_iter(y, tx, batch_size = 1, shuffle=True):
            gamma = 1./pow(np.float(1+n_iter), r)
            if n_iter%10000 == 0:
                print( "Iteration #{}, gamma = {}".format(n_iter, gamma))
            w -= gamma*(calculate_stoch_gradient(b_y, b_tx, w) + lambda_*w)
            
            n_iter +=1
            if n_iter >= max_iters or gamma <= 1e-5:
                break
        if n_iter >= max_iters or gamma <= 1e-5:
            break
    return w, log_loss(y,tx,w)



In [12]:
"""some helper functions for project 1."""
import csv


def load_csv_data(data_path, sub_sample=False):
    """Loads data and returns y (class labels), tX (features) and ids (event ids)"""
    y = np.genfromtxt(data_path, delimiter=",", skip_header=1, dtype=str, usecols=1)
    x = np.genfromtxt(data_path, delimiter=",", skip_header=1)
    ids = x[:, 0].astype(np.int)
    input_data = x[:, 2:]

    # convert class labels from strings to binary (-1,1)
    yb = np.ones(len(y))
    yb[np.where(y=='b')] = 0.
    
    
    # sub-sample
    if sub_sample:
        yb = yb[::50]
        input_data = input_data[::50]
        ids = ids[::50]

    return yb, input_data, ids


def predict_labels(weights, data):
    """Generates class predictions given weights, and a test data matrix"""
    y_pred = np.dot(data, weights)
    y_pred[np.where(y_pred <= 0)] = -1
    y_pred[np.where(y_pred > 0)] = 1
    
    return y_pred


def create_csv_submission(ids, y_pred, name):
    """
    Creates an output file in csv format for submission to kaggle
    Arguments: ids (event ids associated with each prediction)
               y_pred (predicted class labels)
               name (string name of .csv output file to be created)
    """
    with open(name, 'w') as csvfile:
        fieldnames = ['Id', 'Prediction']
        writer = csv.DictWriter(csvfile, delimiter=",", fieldnames=fieldnames)
        writer.writeheader()
        for r1, r2 in zip(ids, y_pred):
              writer.writerow({'Id':int(r1),'Prediction':int(r2)})


In [13]:
# Useful starting lines
%matplotlib inline
import matplotlib.pyplot as plt
path ='data/'
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
DATA_TRAIN_PATH = path + 'train.csv' # TODO: download train data and supply path here 
y, tX, _ = load_csv_data(DATA_TRAIN_PATH)
tX = replace_missing_values(tX)
tX = normalize_data(tX)
tX,y,_ = remove_outliers(tX, y, threshold=5.0)

In [25]:
N,d = tX.shape
w0 = 0.001*np.ones((d,1))

#weights, losses = ridge_regression(y, tX, lambda_ = 1e-15) # param seven is chosen as the best from np.logspace(-15, -5, 10)
#weights,loss = least_squares_GD(y, tX, w0, max_iters = 1000, gamma = 0.7)
#weights,loss = least_squares(y,tX)
#weights,loss = least_squares_SGD(y, tX, w0, max_iters = N, gamma = 0.7)
weights, loss = logistic_regression(y, tX, initial_w = w0, max_iters = 1.1 * N, gamma = 0.7)
print(loss)

Iteration #0, gamma = 1.0
Iteration #10000, gamma = 0.0009999250065618984
Iteration #20000, gamma = 0.0005945812608434309
Iteration #30000, gamma = 0.0004386803706872588
Iteration #40000, gamma = 0.00035354676161220895
Iteration #50000, gamma = 0.0002990652702764048
Iteration #60000, gamma = 0.000260844169466889
Iteration #70000, gamma = 0.00023236559061565857
Iteration #80000, gamma = 0.00021022213298401132
Iteration #90000, gamma = 0.0001924484859947194
Iteration #100000, gamma = 0.0001778266073060046
Iteration #110000, gamma = 0.0001655588972667897
Iteration #120000, gamma = 0.0001550998404773567
Iteration #130000, gamma = 0.0001460629205698025
Iteration #140000, gamma = 0.0001381661469868498
Iteration #150000, gamma = 0.00013119865542496488
Iteration #160000, gamma = 0.00012499941406570435
Iteration #170000, gamma = 0.00011944318980213427
Iteration #180000, gamma = 0.00011443103119920765
Iteration #190000, gamma = 0.00010988365203480858
Iteration #200000, gamma = 0.0001057367298315

In [16]:
del y, tX, w0

In [26]:
DATA_TEST_PATH = path + 'test.csv' # TODO: download train data and supply path here 
y_test, tX_test,_ = load_csv_data(DATA_TEST_PATH)
y_pred = predict_labels(weights, normalize_data(tX_test))

In [27]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.13513703764971718

In [28]:
y_pred[:10]

array([[-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [ 1.]])

In [29]:
OUTPUT_PATH = path + 'log_reg.csv' # TODO: fill in desired name of output file for submission
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)


In [ ]:
print("ridge_loss = ", loss)

In [ ]:
print("SGD_loss = ", loss)

In [ ]:
print("Least_squares_loss = ", loss)

### **Ridge regression**

## Generate predictions and save ouput in csv format for submission:

In [ ]:
acc = 0.
for i in range(len(y_pred)):
  if y_pred[i] == y_test[i]:
    acc += 1
print(acc / len(y_pred))

In [ ]:
df = pd.read_csv('../../data/video_games_sales.csv')
df.info()
cols = ['Global_Sales', 'Critic_Score', 'Critic_Count', 'User_Score', 'User_Count']
sns_plot = sns.pairplot(df[cols])
sns_plot.savefig('pairplot.png')

In [ ]:
def build_poly(x, degree):
    """polynomial basis functions for input data x, for j=0 up to j=degree."""
    if degree == 1:
      return x
    N,d = x.shape
    ext_x = x
    
    for j in range(2, degree + 1):
         ext_x = np.concatenate((ext_x, x[:,1:]**j), axis = 1)
    return ext_x
  
def plot_train_test(train_errors, test_errors, lambdas):
    """
    train_errors, test_errors and lambas should be list (of the same size) the respective train error and test error for a given lambda,
    * lambda[0] = 1
    * train_errors[0] = RMSE of a ridge regression on the train set
    * test_errors[0] = RMSE of the parameter found by ridge regression applied on the test set
    
    degree is just used for the title of the plot.
    """
    plt.semilogx(lambdas, train_errors, color='b', marker='*', label="Train error")
    plt.semilogx(lambdas, test_errors, color='r', marker='*', label="Test error")
    plt.xlabel("lambda")
    plt.ylabel("RMSE")
    leg = plt.legend(loc=1, shadow=True)
    leg.draw_frame(False)
    plt.savefig("ridge_regression")
"""ridge regression demo."""
lambdas = np.logspace(-15, -5, 10)

rmse_tr = []
rmse_te = []
w = []
ratio = len(y)/len(y_test)

for ind, lambda_ in enumerate(lambdas):
    weights, mse = ridge_regression(y, tX, lambda_)
    w.append(weights)
    rmse_tr.append(np.sqrt(mse*2.))
    rmse_te.append(loss_function(y_test,preprocess_data(tX_test),weights))
    print("proportion={p}, lambda={l}, Training MSE={tr:.3f}, Testing MSE={te:.3f}".format(
           p=ratio, l=lambda_, tr=rmse_tr[ind], te=rmse_te[ind]))

# Plot the obtained results
plot_train_test(rmse_tr, rmse_te, lambdas)
plt.show()

In [ ]:
import pandas as pd
import numpy as np

df = pd.DataFrame(preprocess_data(tX)[:,1:])
corr = df.corr()
corr.style.background_gradient(cmap='coolwarm')